In [11]:
pip install psycopg2


  Obtaining dependency information for psycopg2 from https://files.pythonhosted.org/packages/37/2c/5133dd3183a3bd82371569f0dd783e6927672de7e671b278ce248810b7f7/psycopg2-2.9.9-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --- ------------------------------------ 0.1/1.2 MB 1.6 MB/s eta 0:00:01
   ------------------- -------------------- 0.6/1.2 MB 4.9 MB/s eta 0:00:01
   ---------------------------------------  1.2/1.2 MB 8.1 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 7.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import psycopg2


In [25]:
df_claustro = pd.read_csv("claustro.csv", sep = ";")
df_DS_clase1 = pd.read_csv("listado_DS_clase_1.csv", sep = ";")
df_DS_clase2 = pd.read_csv("listado_DS_clase_2.csv", sep = ";")
df_FS_clase1 = pd.read_csv("listado_FS_clase_1.csv", sep = ";")
df_FS_clase2 = pd.read_csv("listado_FS_clase_2.csv", sep = ";")
verticales = pd.read_csv("verticales_sql.csv", sep =",", index_col=0)
alumno10 = pd.read_csv("alumno.csv", sep =",")

In [ ]:
alumno10.head()

In [ ]:
verticales.head()

In [27]:
alumnos10_ = pd.merge(alumno10,verticales,right_on="rama",left_on= "clase", how = "inner")

In [31]:
alumnos_100 = alumnos10_.drop(columns=["clase","fecha_inicio","rama","promocion","campus"])

In [36]:
alumnos_100.drop_duplicates(subset=["Nombre","Email"],inplace=True)

In [38]:
alumnos_100.shape

(52, 3)

In [ ]:
alumnos_100.to_csv("alumnos.csv")

In [ ]:
alumnos10_

In [ ]:
df_claustro.head(8)

In [15]:
df_verticales_ = pd.merge(df_claustro,verticales,right_on=["rama","promocion"],left_on=["Vertical","Promocion"], how = "inner")

In [19]:
df_verticales_2 = df_verticales_.drop(columns=["Vertical","Promocion","Campus","fecha_inicio"])

In [21]:
df_verticales_2 = df_verticales_2.drop(columns=["campus"])

In [22]:
df_verticales_2.to_csv("verticales_2.csv")

In [ ]:
df_DS_clase2.head()

In [5]:
df_DS_juntos = pd.merge(df_DS_clase1,df_DS_clase2, how = "outer")

In [6]:
df_FS_juntos = pd.merge(df_FS_clase1,df_FS_clase2, how = "outer")

In [7]:
df_alumnos = pd.merge(df_DS_juntos,df_FS_juntos, how="outer")

In [ ]:
df_alumnos.head()

In [ ]:
df_DS_juntos.head(5)

In [72]:
df_DS_juntos_ = df_DS_juntos.drop(columns=["Nombre","Email","Promoción","Fecha_comienzo","Campus"])

In [ ]:
df_DS_juntos_.head()

In [74]:
df_DS_juntos_.to_csv("Expediente_ds.csv", index = False)

In [ ]:
df_FS_clase1.head()

In [ ]:
df_FS_clase2.head()

In [ ]:
df_alumnos = df_alumnos.drop(columns=['Proyecto_HLF', 'Proyecto_EDA', 'Proyecto_BBDD', 'Proyecto_ML', 'Proyecto_Deployment', 'Proyecto_WebDev', 'Proyecto_FrontEnd', 'Proyecto_Backend', 'Proyecto_React', 'Proyecto_FullSatck'])

In [ ]:
df_alumnos["clase"] = df_alumnos.index < 25
df_alumnos["clase"] = df_alumnos["clase"].apply(lambda x: "DS" if x else "FS")

In [ ]:
df_alumnos_ = df_alumnos.drop(columns=["Nombre", "Email"])

In [ ]:
df_alumnos_2 = df_alumnos.drop(columns=["Promoción", "Fecha_comienzo","Campus"])

In [ ]:
from datetime import datetime

# Define una función para convertir una cadena de fecha en un objeto de fecha
def convertir_fecha(fecha_str):
    return datetime.strptime(fecha_str, '%d/%m/%Y')

# Aplica la función a todas las filas de la columna "Fecha_comienzo"
df_alumnos['Fecha_comienzo'] = df_alumnos['Fecha_comienzo'].apply(convertir_fecha)

In [32]:
# Datos de conexión de la BBDD en render
host = "dpg-cl14h2q35nfs73866ji0-a.Oregon-postgres.render.com"
port = 5432
database = "prueba1_db_thebridge"
user = "prueba1_db_thebridge_user"
password = "0kdWzE8B4J1EdlTgBCCfvJBmW5YEq6fT"

In [33]:
# Intentamos conectar a la base de datos
conn = psycopg2.connect(
    host=host,
    port=port,
    database=database,
    user=user,
    password=password
)
print("Conexión exitosa a la base de datos en Render")

Conexión exitosa a la base de datos en Render


In [34]:
# Una vez conectada, creamos la tabla

cursor = conn.cursor()

cursor.execute("""
            CREATE TABLE "Profesores" (
                "Id_profesor" SERIAL PRIMARY KEY,
                "Nombre" VARCHAR(255) NOT NULL,
                "Vertical" VARCHAR(255) NOT NULL,
                "Rol" VARCHAR(255) NOT NULL,
                "Promoción" VARCHAR(255) NOT NULL,
                "Id_Campus" VARCHAR(255) NOT NULL)
        """)
# Confirmamos la creación de la tabla. El commit es importante porque la Base de datos no está en local
conn.commit()

# Cerramos la conexión
conn.close()

DuplicateTable: relation "Profesores" already exists


In [10]:
# Volvemos a crear la conexión 

conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )

cursor = conn.cursor()

# Iteramos por las columnas y metemos los datos del Df en las columnas correspondientes
for index, row in df_claustro.iterrows():
    cursor.execute("""
        INSERT INTO "Profesores" ("Nombre", "Rol", "Vertical", "Promoción", "Id_Campus")
        VALUES (%s, %s, %s, %s, %s)
    """, (row['Nombre'], row['Rol'], row['Vertical'], row['Promocion'], row['Campus']))
conn.commit()
cursor.close()

In [9]:
# Para hacer consulta de los datos subidos a SQL
conn = psycopg2.connect(
            host=host,
            port=port,
            database=database,
            user=user,
            password=password
        )
cursor = conn.cursor()

cursor.execute('SELECT * FROM "Profesores"')

# Recupera los resultados de la consulta
rows = cursor.fetchall()
pd.DataFrame(rows)

,0,1,2,3,4,5
0,1,Noa Yáñez,DS,TA,Septiembre,Madrid
1,2,Saturnina Benitez,DS,TA,Septiembre,Madrid
2,3,Anna Feliu,FS,TA,Septiembre,Madrid
3,4,Rosalva Ayuso,FS,TA,Septiembre,Valencia
4,5,Ana Sofía Ferrer,FS,TA,Febrero,Valencia
5,6,Angélica Corral,FS,TA,Febrero,Madrid
6,7,Ariel Lledó,DS,TA,Septiembre,Madrid
7,8,Mario Prats,FS,LI,Febrero,Valencia
8,9,Luis Ángel Suárez,FS,LI,Septiembre,Madrid
9,10,María Dolores Diaz,DS,LI,Septiembre,Madrid


In [62]:
conn = psycopg2.connect(
            host=host,
            port=port,
            database=database,
            user=user,
            password=password
        )
cursor = conn.cursor()

cursor.execute("""
            CREATE TABLE "Alumno" (
                "Id_alumno" SERIAL PRIMARY KEY,
                "Nombre" VARCHAR(255) NOT NULL,
                "Email" VARCHAR(255) NOT NULL,
                "Promoción" VARCHAR(255) NOT NULL,
                "Fecha_comienzo" DATE NOT NULL,
                "Id_Campus" VARCHAR(255) NOT NULL,
                "Vertical" VARCHAR(255) NOT NULL)
        """)
# Confirmamos la creación de la tabla. El commit es importante porque la Base de datos no está en local
conn.commit()

# Cerramos la conexión
conn.close()




In [19]:
df_alumnos_ = df_alumnos.drop(columns=["Nombre", "Email"])

In [13]:
df_alumnos_2 = df_alumnos.drop(columns=["Promoción", "Fecha_comienzo","Campus"])

In [16]:
df_alumnos_.head()

,Promoción,Fecha_comienzo,Campus,clase
0,Septiembre,18/09/2023,Madrid,DS
1,Septiembre,18/09/2023,Madrid,DS
2,Septiembre,18/09/2023,Madrid,DS
3,Septiembre,18/09/2023,Madrid,DS
4,Septiembre,18/09/2023,Madrid,DS


In [ ]:
verticales

In [20]:
verticales.head()

,Unnamed: 0,vertical_id,rama,promocion,fecha_inicio,campus
0,0,3,DS,Septiembre,2023-09-18,1
1,1,4,DS,Septiembre,2023-09-18,1
2,2,5,DS,Septiembre,2023-09-18,1
3,3,6,DS,Septiembre,2023-09-18,1
4,4,7,DS,Septiembre,2023-09-18,1


In [59]:
df_alumnos_2.to_csv("alumno.csv", index = False)

In [20]:
df_alumnos_.head()

,Promoción,Fecha_comienzo,Campus,clase
0,Septiembre,2023-09-18,Madrid,DS
1,Septiembre,2023-09-18,Madrid,DS
2,Septiembre,2023-09-18,Madrid,DS
3,Septiembre,2023-09-18,Madrid,DS
4,Septiembre,2023-09-18,Madrid,DS


In [46]:
df_alumnos_.to_csv("vertical.csv", index = False)

In [24]:
verticales_ = verticales.drop(columns=["Unnamed: 0","vertical_id"])

In [25]:
verticales_.head()

,rama,promocion,fecha_inicio,campus
0,DS,Septiembre,2023-09-18,1
1,DS,Septiembre,2023-09-18,1
2,DS,Septiembre,2023-09-18,1
3,DS,Septiembre,2023-09-18,1
4,DS,Septiembre,2023-09-18,1


In [22]:
verticales.head()

,Unnamed: 0,vertical_id,rama,promocion,fecha_inicio,campus
0,0,3,DS,Septiembre,2023-09-18,1
1,1,4,DS,Septiembre,2023-09-18,1
2,2,5,DS,Septiembre,2023-09-18,1
3,3,6,DS,Septiembre,2023-09-18,1
4,4,7,DS,Septiembre,2023-09-18,1


In [64]:
conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )

cursor = conn.cursor()

# Iteramos por las columnas y metemos los datos del Df en las columnas correspondientes
for index, row in df_alumnos.iterrows():
    cursor.execute("""
        INSERT INTO "Alumno" ("Nombre", "Email", "Promoción", "Fecha_comienzo", "Id_Campus", "Vertical")
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (row['Nombre'], row['Email'], row['Promoción'], row['Fecha_comienzo'], row['Campus'], row["clase"]))
conn.commit()
cursor.close()

In [83]:
# Para hacer consulta de los datos subidos a SQL
conn = psycopg2.connect(
            host=host,
            port=port,
            database=database,
            user=user,
            password=password
        )
cursor = conn.cursor()

cursor.execute('SELECT * FROM "Alumno"')

# Recupera los resultados de la consulta
rows = cursor.fetchall()
pd.DataFrame(rows)

,0,1,2,3,4,5,6
0,1,Jafet Casals,Jafet_Casals@gmail.com,Septiembre,2023-09-18,Madrid,DS
1,2,Jorge Manzanares,Jorge_Manzanares@gmail.com,Septiembre,2023-09-18,Madrid,DS
2,3,Onofre Adadia,Onofre_Adadia@gmail.com,Septiembre,2023-09-18,Madrid,DS
3,4,Merche Prada,Merche_Prada@gmail.com,Septiembre,2023-09-18,Madrid,DS
4,5,Pilar Abella,Pilar_Abella@gmail.com,Septiembre,2023-09-18,Madrid,DS
5,6,Leoncio Tena,Leoncio_Tena@gmail.com,Septiembre,2023-09-18,Madrid,DS
6,7,Odalys Torrijos,Odalys_Torrijos@gmail.com,Septiembre,2023-09-18,Madrid,DS
7,8,Eduardo Caparrós,Eduardo_Caparrós@gmail.com,Septiembre,2023-09-18,Madrid,DS
8,9,Ignacio Goicoechea,Ignacio_Goicoechea@gmail.com,Septiembre,2023-09-18,Madrid,DS
9,10,Clementina Santos,Clementina_Santos@gmail.com,Septiembre,2023-09-18,Madrid,DS


In [69]:
df_DS_juntos.head()

,Nombre,Promoción,Fecha_comienzo,Campus,Proyecto_HLF,Proyecto_EDA,Proyecto_BBDD,Proyecto_ML,Proyecto_Deployment
0,Jafet Casals,Septiembre,18/09/2023,Madrid,Apto,No Apto,Apto,Apto,Apto
1,Jorge Manzanares,Septiembre,18/09/2023,Madrid,Apto,No Apto,Apto,Apto,Apto
2,Onofre Adadia,Septiembre,18/09/2023,Madrid,Apto,Apto,Apto,No Apto,Apto
3,Merche Prada,Septiembre,18/09/2023,Madrid,Apto,No Apto,No Apto,Apto,No Apto
4,Pilar Abella,Septiembre,18/09/2023,Madrid,Apto,No Apto,Apto,Apto,Apto


In [68]:
df_DS_juntos = df_DS_juntos.drop(columns=["Email"])

In [ ]:
conn = psycopg2.connect(
            host=host,
            port=port,
            database=database,
            user=user,
            password=password
        )
cursor = conn.cursor()

cursor.execute("""
            CREATE TABLE "Data Science" (
                "Id_DC" SERIAL PRIMARY KEY,
                "Nombre" VARCHAR(255) NOT NULL,
                "Promocion" VARCHAR(255) NOT NULL,
                "Fecha_comienzo" DATE NOT NULL,
                "id_Campus" VARCHAR(255) NOT NULL,
                "Id_profesor" VARCHAR(255) REFERENCES profesores(Id) CHECK (Id_profesor IN (SELECT Id FROM profesores WHERE Vertical = 'DS')),
                "Proyect_HLF" VARCHAR(255) NOT NULL,
                "Proyect_EDA" VARCHAR(255) NOT NULL,
                "Proyect_BBDD" VARCHAR(255) NOT NULL,
                "Proyect_ML" VARCHAR(255) NOT NULL,
                "Proyect_Deployment" VARCHAR(255) NOT NULL)
        """)

In [ ]:

conn = psycopg2.connect(
            host=host,
            port=port,
            database=database,
            user=user,
            password=password
        )
cursor = conn.cursor()

cursor.execute("""
            CREATE TABLE "Data Science" (
                "Id_DC" SERIAL PRIMARY KEY,
                "Nombre" VARCHAR(255) NOT NULL,
                "Promocion" VARCHAR(255) NOT NULL,
                "Fecha_comienzo" DATE NOT NULL,
                "id_Campus" VARCHAR(255) NOT NULL,
                "Id_profesor" VARCHAR(255) REFERENCES profesores(Id),
                "Proyect_HLF" VARCHAR(255) NOT NULL,
                "Proyect_EDA" VARCHAR(255) NOT NULL,
                "Proyect_BBDD" VARCHAR(255) NOT NULL,
                "Proyect_ML" VARCHAR(255) NOT NULL,
                "Proyect_Deployment" VARCHAR(255) NOT NULL)
                CHECK (Id_profesor IN (SELECT Id FROM profesores WHERE Vertical = 'DS'))
        """)

conn.commit()
conn.close()

In [70]:
df_DS_juntos.head()

,Nombre,Promoción,Fecha_comienzo,Campus,Proyecto_HLF,Proyecto_EDA,Proyecto_BBDD,Proyecto_ML,Proyecto_Deployment
0,Jafet Casals,Septiembre,18/09/2023,Madrid,Apto,No Apto,Apto,Apto,Apto
1,Jorge Manzanares,Septiembre,18/09/2023,Madrid,Apto,No Apto,Apto,Apto,Apto
2,Onofre Adadia,Septiembre,18/09/2023,Madrid,Apto,Apto,Apto,No Apto,Apto
3,Merche Prada,Septiembre,18/09/2023,Madrid,Apto,No Apto,No Apto,Apto,No Apto
4,Pilar Abella,Septiembre,18/09/2023,Madrid,Apto,No Apto,Apto,Apto,Apto


In [ ]:
conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )

cursor = conn.cursor()

# Iteramos por las columnas y metemos los datos del Df en las columnas correspondientes
for index, row in df_DS_clase1.iterrows():
    cursor.execute("""
        INSERT INTO "Data Science" ("Nombre", "Promoción", "Fecha_comienzo", "Campus", "Proyecto_HLF", "Proyecto_EDA", "Proyecto_BBDD","Proyecto_ML", "Proyecto_Deployment")
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (row['Nombre'], row['Promoción'], row['Fecha_comienzo'], row['Campus'], row['Proyecto_HLF'], row["Proyecto_EDA"], row["Proyecto_BBDD"], row["Proyecto_ML"], row["Proyecto_Deployment"]))
conn.commit()
cursor.close()

In [61]:
"""
conn = psycopg2.connect(
            host=host,
            port=port,
            database=database,
            user=user,
            password=password
        )
cursor = conn.cursor()
cursor.execute('''
                 DROP TABLE "Alumno"
                 ''')
conn.commit()
cursor.close()
"""


In [ ]:
df_DS_juntos = df_DS_juntos.mask(df_DS_juntos.columns != "Email")

In [ ]:

conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )

cursor = conn.cursor()

# Iteramos por las columnas y metemos los datos del Df en las columnas correspondientes
for index, row in df_DS_clase1.iterrows():
    cursor.execute("""
        INSERT INTO "Data Science" ("Nombre", "Promoción", "Fecha_comienzo", "Campus", "Proyecto_HLF", "Proyecto_EDA", "Proyect_BBDD","Proyect_ML", "Proyect_Deployment",")
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,)
    """, (row['Nombre'], row['Rol'], row['Vertical'], row['Promocion'], row['Campus']))

ya s
conn.commit()
cursor.close()



In [47]:
df_alumnos.head()

,Nombre,Email,Promoción,Fecha_comienzo,Campus,clase
0,Jafet Casals,Jafet_Casals@gmail.com,Septiembre,2023-09-18,Madrid,DS
1,Jorge Manzanares,Jorge_Manzanares@gmail.com,Septiembre,2023-09-18,Madrid,DS
2,Onofre Adadia,Onofre_Adadia@gmail.com,Septiembre,2023-09-18,Madrid,DS
3,Merche Prada,Merche_Prada@gmail.com,Septiembre,2023-09-18,Madrid,DS
4,Pilar Abella,Pilar_Abella@gmail.com,Septiembre,2023-09-18,Madrid,DS
